In [21]:
# to upload the dataset
from google.colab import files
uploaded = files.upload()

Saving Sentiment.csv to Sentiment (1).csv


In [22]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re 
#Package for splitting the data
from sklearn.model_selection import train_test_split 
#Package for conversion of categorical to Numerical values
from sklearn.preprocessing import LabelEncoder 
#Tokenization
from keras.preprocessing.text import Tokenizer 
#Add zeros or crop based on the length
from keras.preprocessing.sequence import pad_sequences 
#Sequential Neural Network
from keras.models import Sequential 
#For layers in Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D 
from keras.utils.np_utils import to_categorical

In [23]:
#Loading the dataset
data = pd.read_csv('Sentiment.csv') 
#Keeping only the neccessary columns
data = data[['text','sentiment']] 

In [24]:
#converting values to lower case
data['text'] = data['text'].apply(lambda x: x.lower()) 
#only a-z,A-Z,0-9 would be remaining in the data, else special characters are removed using regular expression
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', '', x)))

In [25]:
#Fpr Removing Retweets
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') 

In [26]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [27]:
X = pad_sequences(X)
embed_dim = 128
lstm_out = 196

In [28]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

In [29]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [30]:
batch_size = 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 26s - loss: 0.8233 - accuracy: 0.6426
0.7745055123519148
0.6653560400009155


In [31]:
print(model.metrics_names)

['loss', 'accuracy']


## Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [32]:
model.save('sentimentAnalysis.h5')

In [33]:
from keras.models import load_model
model= load_model('sentimentAnalysis.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [34]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [35]:
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 0:
  print("Neutral")
elif sentiment < 0:
  print("Negative")
elif sentiment > 0:
  print("Positive")
else:
  print("Can not be determined")

0
Neutral


## Apply GridSearchCV on the source code provided in the class

In [36]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20, 40]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train,Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 37s - loss: 0.8346 - accuracy: 0.6388
Epoch 1/1
 - 38s - loss: 0.8262 - accuracy: 0.6443
Epoch 1/1
 - 38s - loss: 0.8348 - accuracy: 0.6418
Epoch 1/1
 - 38s - loss: 0.8291 - accuracy: 0.6425
Epoch 1/1
 - 38s - loss: 0.8237 - accuracy: 0.6465
Epoch 1/2
 - 38s - loss: 0.8328 - accuracy: 0.6505
Epoch 2/2
 - 38s - loss: 0.6897 - accuracy: 0.7053
Epoch 1/2
 - 37s - loss: 0.8226 - accuracy: 0.6501
Epoch 2/2
 - 37s - loss: 0.6845 - accuracy: 0.7066
Epoch 1/2
 - 38s - loss: 0.8328 - accuracy: 0.6419
Epoch 2/2
 - 38s - loss: 0.6796 - accuracy: 0.7089
Epoch 1/2
 - 37s - loss: 0.8292 - accuracy: 0.6472
Epoch 2/2
 - 37s - loss: 0.6813 - accuracy: 0.7071
Epoch 1/2
 - 37s - loss: 0.8245 - accuracy: 0.6420
Epoch 2/2
 - 37s - loss: 0.6798 - accuracy: 0.7107
Epoch 1/1
 - 26s - loss: 0.8306 - accuracy: 0.6412
Epoch 1/1
 - 27s - loss: 0.8310 - accuracy: 0.6406
Epoch 1/1
 - 26s - loss: 0.8366 - accuracy: 0.6360
Epoch 1/1
 - 27s - loss: 0.8352 - accuracy: 0.6332
Epoch 1/1
 - 26s - loss: 0.8272